In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
import FormulaSlicer
import os
import scipy.ndimage as im
from importlib import reload
import tables

In [4]:
# load STL mesh and get X,Y, Z range
stlName='/home/sam/ULB/Mémorants/Marco/new_connector.stl'
rootName='MarcoConnector'

lowresSlicing=1
lowresHashing=1

mesh,Zmins,Zmaxs=FormulaSlicer.loadStl(stlName,centred=True,mulDimension=100)
X,Y,Z=FormulaSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} -> {}  Y: {} -> {}  Z: {} -> {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)



X: -1265 -> 1265  Y: -755 -> 755  Z: 0.0 -> 1221.0
(1511, 2531) (1511, 2531) (1222,)


In [5]:
Zmins.max(),Zmaxs.max(),Zmins.min()

(1219.9866, 1220.0, 0.0)

In [6]:
# Do a first low resolution full scale stack with a correct infill
lowresH5=rootName+'_lowres.h5'
FormulaSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

In [7]:
FOVrad=150  # more or less standard value but should be input
FOVz=250
overlapZ=2

amZ=int((Z.max()-Z.min())/FOVz)+1
Zrange=(Z.max()-Z.min())/amZ
print(amZ,Zrange)

# loop over different Z-slices
centreCoords=[]
for k in range(amZ):
    Zstart=k*Zrange
    Zstop=(k+1)*Zrange+overlapZ

    # now get low-resolution projection figures for this Z-range
    Ground,X,Y=FormulaSlicer.getProjection(lowresH5,Zstart,Zstop)

    # doOptimalStitching or hexagonal/square and find blockcentres
    optPoints=FormulaSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

    centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


5 244.2


In [21]:
clf()
op_pix=FormulaSlicer.convertToPixels(X,Y,centreCoords[-1][:,:2])
plot(op_pix[:,0],op_pix[:,1],'r.-')
imshow(Ground)

In [8]:
ordCentres=[]
for optPoints in centreCoords:
    FormulaSlicer.writeTSPfile('blocks.tsp',optPoints[:,:2])
    !concorde -o 'tsp.sol' 'blocks.tsp'
    Ord=FormulaSlicer.readTSPfile('tsp.sol')
    ordCentres.append(optPoints[Ord])

concorde -o tsp.sol blocks.tsp
Host: lapsam  Current process id: 10138
Using random seed 1505802748
Problem Name: blocks
Problem Type: TSP
Nanoscribe Blocks in Ham Path
Number of Nodes: 55
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 12250 (from tour)
  LP Value  1: 12182.000000  (0.00 seconds)
  LP Value  2: 12223.000000  (0.00 seconds)
  LP Value  3: 12250.000000  (0.01 seconds)
New lower bound: 12250.000000
Final lower bound 12250.000000, upper bound 12250.000000
Exact lower bound: 12250.000000
DIFF: 0.000000
Final LP has 61 rows, 158 columns, 372 nonzeros
Optimal Solution: 12250.00
Number of bbnodes: 1
Total Running Time: 0.04 (seconds)
concorde -o tsp.sol blocks.tsp
Host: lapsam  Current process id: 10142
Using random seed 1505802748
Problem Name: blocks
Problem Type: TSP
Nanoscribe Blocks in Ham Path
Number of Nodes: 60
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 13552 (from tour)
  LP Value  1: 13473.000000  (0.00 seconds)
  LP Value  2: 13552.000000

In [8]:
clf()
op_pix=FormulaSlicer.convertToPixels(X,Y,ordCentres[-1][:,:2])
plot(op_pix[:,0],op_pix[:,1],'r.-')
imshow(Ground)

# now try to have a TSP working over multiple heights

In [9]:
Zgrower=1000
newPPlist=[]
for cc in centreCoords:
    newPPlist.extend(cc*np.array([1,1,Zgrower]))
newPP=np.asarray(newPPlist)

FormulaSlicer.writeTSPfile('blocks.tsp',newPP)
!concorde -o 'tsp.sol' 'blocks.tsp'
Ord=FormulaSlicer.readTSPfile('tsp.sol')
ord3DCentres=(newPP[Ord]*np.array([1,1,1/Zgrower]))

if (ord3DCentres[0,2]>ord3DCentres[-1,2]):
    ord3DCentres=ord3DCentres[::-1]

concorde -o tsp.sol blocks.tsp
Host: lapsam  Current process id: 10146
Using random seed 1505802758
Problem Name: blocks
Problem Type: TSP
Nanoscribe Blocks in Ham Path
Number of Nodes: 277
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 1040843 (from tour)
  LP Value  1: 64311.000000  (0.05 seconds)
  LP Value  2: 1040729.000000  (0.10 seconds)
  LP Value  3: 1040850.788609  (0.79 seconds)
New lower bound: 1040508.266667
Final lower bound 1040508.266667, upper bound 1040843.000000
Exact lower bound: 1040508.266667
DIFF: 0.000000
Time for Total: 0.95 seconds (0.95 total in 1 calls)
Final LP has 368 rows, 848 columns, 4749 nonzeros
LOWER BOUND: 1040508.266667   ACTIVE NODES: 1

Task 0: Branching on node 0
BBnode 0 split into 1 (1040564.02) 2 (1040533.27) (0.09 seconds)
LOWER BOUND: 1040533.266667   ACTIVE NODES: 2

Task 1: Cutting on node 2
New upperbound from x-heuristic: 1040807.00
TOUR FOUND - upperbound is 1040807.00
Writing Pool: 139 cuts
BBnode 2 (now 3) can be pruned: 

In [10]:
ord3DCentres

array([[  1.02500000e+03,   1.95000000e+02,   0.00000000e+00],
       [  9.03000000e+02,   1.04000000e+02,   0.00000000e+00],
       [  7.76000000e+02,   1.95000000e+02,   0.00000000e+00],
       [  5.53000000e+02,   2.05000000e+02,   0.00000000e+00],
       [  3.30000000e+02,   1.95000000e+02,   0.00000000e+00],
       [  1.24000000e+02,   2.20000000e+02,   0.00000000e+00],
       [ -7.20000000e+01,   2.05000000e+02,   0.00000000e+00],
       [ -7.00000000e+01,   4.20000000e+02,   0.00000000e+00],
       [  1.98000000e+02,   4.32000000e+02,   0.00000000e+00],
       [  4.25000000e+02,   4.45000000e+02,   0.00000000e+00],
       [  6.59000000e+02,   4.41000000e+02,   0.00000000e+00],
       [  9.02000000e+02,   4.22000000e+02,   0.00000000e+00],
       [  1.13900000e+03,   4.31000000e+02,   0.00000000e+00],
       [  1.02000000e+03,   6.61000000e+02,   0.00000000e+00],
       [  8.99000000e+02,   7.50000000e+02,   0.00000000e+00],
       [  7.78000000e+02,   6.60000000e+02,   0.0000000

In [40]:
clf()
scatter(ord3DCentres[:,0],ord3DCentres[:,1],s=ord3DCentres[:,2])
plot(ord3DCentres[:,0],ord3DCentres[:,1],'r-')

# now continue creating blocks

In [43]:
zstacks=np.unique(ord3DCentres[:,2])
zstacks

array([   0.,  243.,  486.,  729.])

In [57]:
stackI=np.argwhere(np.abs(ord3DCentres[:,2]-243)<1)[:,0]
optPoints=ord3DCentres[stackI,:2]

In [58]:
optPoints=ord3DCentres[stackI,:2]
optPoints.shape

(11, 2)

In [11]:
slicing=1

hashing=1
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing

if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
lowresH5=rootName+'_lowres.h5'


# now creates the stacks
zstacks=np.unique(ord3DCentres[:,2])



for k in np.arange(len(zstacks)):
    Zstart=zstacks[k]
    Zstop=Zstart+Zrange
    zz=np.arange(Zstart,Zstop+slicing,slicing)
    sliceIndList= np.arange(len(zz))# adapt for variable slicing

    stackI=np.argwhere(np.abs(ord3DCentres[:,2]-Zstart)<1)[:,0]
    optPoints=ord3DCentres[stackI,:2]
    # generate voronoi polygons
    regions, vertices = FormulaSlicer.makeVoronoiPolygons(optPoints)

    # do loop over blocks and write out the stacks (needs stl and low-res stack) 
    #                                        => CAN I USE formulaToStack and give stl-name and low-res stack as parameters??
    fullDist=[]
    zCentres=[]
    for i in np.arange(len(optPoints)):
#    for i in np.arange(2):
        # now make coordinate systems
        print('Writing block {} in layer {}'.format(i,k))
        xc=optPoints[i,0]
        yc=optPoints[i,1]

        polygon = vertices[regions[i]]

        am=np.int(np.round(2*FOVrad/hashing))+1
        xx=np.linspace(xc-FOVrad,xc+FOVrad,am)
        yy=np.linspace(yc-FOVrad,yc+FOVrad,am)
        Xloc,Yloc=np.meshgrid(xx,yy)

        MM=FormulaSlicer.makeVoronoiMask(polygon,Xloc,Yloc)
        # if you need overlap, do binary_dilation on the mask
        MM=im.binary_dilation(MM,iterations=overlapXY_multiplier) # overlap = 2*iterations*hatching

        # with these coordinate systems, apply stlToStack
        blockName=genBlockName.format(k,i)

        FormulaSlicer.stlToStack(mesh,blockName,Xloc,Yloc,zz[sliceIndList],Zmins,Zmaxs,inFillMaster=lowresH5,writingMask=MM)

Writing block 0 in layer 0
Writing block 1 in layer 0
Writing block 2 in layer 0
Writing block 3 in layer 0
Writing block 4 in layer 0
Writing block 5 in layer 0
Writing block 6 in layer 0
Writing block 7 in layer 0
Writing block 8 in layer 0
Writing block 9 in layer 0
Writing block 10 in layer 0
Writing block 11 in layer 0
Writing block 12 in layer 0
Writing block 13 in layer 0
Writing block 14 in layer 0
Writing block 15 in layer 0
Writing block 16 in layer 0
Writing block 17 in layer 0
Writing block 18 in layer 0
Writing block 19 in layer 0
Writing block 20 in layer 0
Writing block 21 in layer 0
Writing block 22 in layer 0
Writing block 23 in layer 0
Writing block 24 in layer 0
Writing block 25 in layer 0
Writing block 26 in layer 0
Writing block 27 in layer 0
Writing block 28 in layer 0
Writing block 29 in layer 0
Writing block 30 in layer 0
Writing block 31 in layer 0
Writing block 32 in layer 0
Writing block 33 in layer 0
Writing block 34 in layer 0
Writing block 35 in layer 0
Wr

In [12]:
h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'

amShells=0
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False

doColourH5=False


fp=os.path.split(h5Regex)
if (fp[0]==''):
    fnames,fnumbs=FormulaSlicer.findFiles(h5Regex)
else:
    fnames,fnumbs=FormulaSlicer.findFiles(fp[1],fp[0])

for ff in fnames:
    # should decide on the sliceIndList here. It could differ from the stack, but this is difficult to determine here
    # especially if it needs to be in Z => which Z's are we doing here? Unknown in advance.
    gwlName=ff[:-3]+'.gwl'
    print('Writing gwl: {}'.format(gwlName))
    dists=FormulaSlicer.stackToGwl(ff,gwlName,amShells,ScaffStep,hatchStep
                               ,atAngles=-1,writeHeader=False,writeColourH5=doColourH5,doTopBottom=True)

if (len(dists)>0):
    if (len(fullDist)==0):
        fullDist=dists
    else:
        fullDist=fullDist+dists

# Write out regex-gwl to follow this path.
FormulaSlicer.writeGwlForRegex(fullGwlName,genRegex)#,CodeSpeeds,CodeIntensities)

Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00000.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00001.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00002.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00003.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00004.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00005.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00006.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00007.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00000 - nr 00008.gwl
Writing gwl: /home/

Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00022.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00023.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00024.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00025.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00026.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00027.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00028.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00029.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00001 - nr 00030.gwl
Writing gwl: /home/

Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00039.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00040.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00041.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00042.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00043.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00044.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00045.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00046.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00002 - nr 00047.gwl
Writing gwl: /home/

Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00003.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00004.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00005.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00006.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00007.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00008.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00009.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00010.gwl
Writing gwl: /home/sam/ULB/Notebooks/TipSlicer/MarcoConnector/MarcoConnectorBlock-Layer 00004 - nr 00011.gwl
Writing gwl: /home/